## Notations:<br>
$J$: the set of demand points;<br>
$I$: the set of factory points;<br>
$S$: the possible scenarios set;<br> 
$\xi_j$: demand of the demanding point $j$;<br>
$k_j$: unit cost of waste in demand point $j$;<br>
$t_{ij}$: unit cost from factory $i$ to point $j$;<br>
$p_j$: price for sales in demand point $j$;<br>
$c_i$: production unit cost in factory $i$;<br>
$h_i$: production capacity of factory $i$<br>
## Decision variables:<br>
$w_j$: waste amount in demand point $j$;<br>
$y_j$: sale amount in demand point $j$;<br>
$x_i$: amount of production in factory $i$; <br>
$z_{ij}$: amount of good shipping from factory $i$ to point $j$;

## Two-stage stochastic programming formulation:<br>

\begin{align*}
\min_{x,z} \quad & \sum_{i\in I}c_ix_i +\sum_{i\in I,\ j\in J}z_{ij}t_{ij}+ E_{\xi}(Q(y,w,\xi))\\
 s.t.\quad & x_i=\sum_{j\in J}z_{ij} \quad \quad \forall i\in I\\
& x_i \leq h_i\quad \quad \quad \forall i \in I\\
& x_i \geq 0, z_{ij} \geq 0, \quad \forall i \in I,\ j\in J
\end{align*}


### Where the second stage programming problem would be as follows.<br>
\begin{align*}
Q(y,w,\xi) = & \min_{y,w} \sum_{j\in J}(w_jk_j-p_jy_j) \\
s.t.\\ 
     & y_j\leq \xi_j \quad \quad \quad  \forall j\in J\\
     & \sum_{i\in I}z_{ij}=y_j+w_j \quad \quad \forall j\in J\\
     & \ y_j\geq 0,\ w_j\geq 0,\quad \quad \forall j\in J
\end{align*}

### The extensive form of the problem<br>
$y_{j,s}$: the sale amount in point $j$ when scenario $s$ happens;<br>
$w_{j,s}$: the waste amount in point $j$ when scenario $s$ happens;<br>
$Pr_{s}$: the probability when $s$ happens;<br>
<br>
\begin{align*}
\min_{x,z} \quad & \sum_{i\in I}c_ix_i +\sum_{i\in I,\ j\in J}t_{ij}z_{ij}+ \sum_{s\in S}Pr_{s}\sum_{j\in J}(k_jw_{js}-p_jy_{js})\\
 s.t.\quad & x_i-\sum_{j\in J}z_{ij}=0 \quad \quad \forall i\in I\\
& x_i \leq h_i\quad \quad \quad \forall i \in I\\
& \sum_{i\in I}z_{ij}=y_{js}+w_{js}\quad \forall j \in J\ ,\forall s\in S\\
& y_{js}\leq \xi_{js} \quad \forall j\in J\ ,\forall s\in S_j\\
& x_i \geq 0, z_{ij} \geq 0, \quad \forall i \in I,\ j\in J\\
& y_{js} \geq 0,\ w_{js} \geq 0\quad \quad \forall j\in J,\ \forall s\in S_j
\end{align*}

In [1]:
using JuMP
using LinearAlgebra
using Statistics
using Combinatorics

In [2]:
using Gurobi

function defsolver(m)
    setsolver(m, GurobiSolver())
end

defsolver (generic function with 1 method)

### Solve the extensive form

In [3]:
#Initialize the parameters regarding the problem
c=ones(3)*14    #unit cost for production
t=[2.49 5.21 3.76 4.85 2.07;1.46 2.54 1.83 1.86 4.76;3.26 3.08 2.60 3.76 4.45] # transportation unit
p=ones(5)*24   #price
h=[500,450,650] #capacity
k=ones(5)*4    #unit cost for waste
nFactories=length(c)
nDemandPoint=length(p)
dict1=Dict("Low"=>[150,0.25],"Medium"=>[160,0.50],"High"=>[170,0.25])
dict2=Dict("Low"=>[100,0.25],"Medium"=>[120,0.50],"High"=>[135,0.25])
dict3=Dict("Low"=>[250,0.25],"Medium"=>[270,0.50],"High"=>[300,0.25])
dict4=Dict("Low"=>[300,0.30],"Medium"=>[325,0.40],"High"=>[350,0.30])
dict5=Dict("Low"=>[600,0.30],"Medium"=>[700,0.40],"High"=>[800,0.30])
demand=[150 160 170;
        100 120 135;
        250 270 300;
        300 325 350;
        600 700 800;]
probability = [0.25 0.50 0.25;
               0.25 0.50 0.25;
               0.25 0.50 0.25;
               0.30 0.40 0.30;
               0.30 0.40 0.30]
Probability=[]
push!(Probability,dict1)
push!(Probability,dict2)
push!(Probability,dict3)
push!(Probability,dict4)
push!(Probability,dict5)


5-element Array{Any,1}:
 Dict("Low"=>[150.0, 0.25],"Medium"=>[160.0, 0.5],"High"=>[170.0, 0.25])
 Dict("Low"=>[100.0, 0.25],"Medium"=>[120.0, 0.5],"High"=>[135.0, 0.25])
 Dict("Low"=>[250.0, 0.25],"Medium"=>[270.0, 0.5],"High"=>[300.0, 0.25])
 Dict("Low"=>[300.0, 0.3],"Medium"=>[325.0, 0.4],"High"=>[350.0, 0.3])  
 Dict("Low"=>[600.0, 0.3],"Medium"=>[700.0, 0.4],"High"=>[800.0, 0.3])  

In [4]:
# Enumerate Scenario
function SenariosMaking(k,Senarios::Array = reshape([],0,nDemandPoint+1),temp::Array = ones(1,nDemandPoint+1))    
    if k>0
        for i=1:nFactories
            temp[1,nDemandPoint+1-k]=demand[nDemandPoint+1-k,i]
            temp[1,nDemandPoint+1]=temp[1,nDemandPoint+1]*probability[nDemandPoint+1-k,i]
            Senarios = SenariosMaking(k-1,Senarios,temp)
            if k==1
                Senarios=[Senarios;temp];
            end
            temp[1,nDemandPoint+1]=temp[1,nDemandPoint+1]/probability[nDemandPoint+1-k,i];
        end
    end
    return Senarios
end

SenariosMaking (generic function with 3 methods)

In [5]:
Scenarios=SenariosMaking(5)

243×6 Array{Any,2}:
 150.0  100.0  250.0  300.0  600.0  0.00140625
 150.0  100.0  250.0  300.0  700.0  0.001875  
 150.0  100.0  250.0  300.0  800.0  0.00140625
 150.0  100.0  250.0  325.0  600.0  0.001875  
 150.0  100.0  250.0  325.0  700.0  0.0025    
 150.0  100.0  250.0  325.0  800.0  0.001875  
 150.0  100.0  250.0  350.0  600.0  0.00140625
 150.0  100.0  250.0  350.0  700.0  0.001875  
 150.0  100.0  250.0  350.0  800.0  0.00140625
 150.0  100.0  270.0  300.0  600.0  0.0028125 
 150.0  100.0  270.0  300.0  700.0  0.00375   
 150.0  100.0  270.0  300.0  800.0  0.0028125 
 150.0  100.0  270.0  325.0  600.0  0.00375   
   ⋮                                ⋮         
 170.0  135.0  270.0  350.0  600.0  0.0028125 
 170.0  135.0  270.0  350.0  700.0  0.00375   
 170.0  135.0  270.0  350.0  800.0  0.0028125 
 170.0  135.0  300.0  300.0  600.0  0.00140625
 170.0  135.0  300.0  300.0  700.0  0.001875  
 170.0  135.0  300.0  300.0  800.0  0.00140625
 170.0  135.0  300.0  325.0  600.0  0.00

In [6]:
M2 = Model()
nScenario=size(Scenarios)[1]
@variable(M2,x[1:nFactories]>=0)
@variable(M2,z[1:nFactories,1:nDemandPoint]>=0)
@variable(M2,y[1:nDemandPoint,1:nScenario]>=0)
@variable(M2,w[1:nDemandPoint,1:nScenario]>=0)



#constraints regarding the first stage 

for i=1:nFactories
   @constraint(M2,x[i]-sum(z[i,j] for j in 1:nDemandPoint)==0)
   @constraint(M2,x[i]<=h[i])
end
#constraints regarding the second stage
EQ=0
for s =1:nScenario
   for j= 1:nDemandPoint 
        @constraint(M2,sum(z[i,j] for i in 1:nFactories) == w[j,s] + y[j,s])
        @constraint(M2,y[j,s] <= Scenarios[s,j])
        EQ += Scenarios[s,nDemandPoint+1]*(k[j]*w[j,s]-p[j]*y[j,s])
   end
end
@objective(M2,Min,sum(x[i]*c[i] for i in 1:nFactories) +sum(t[i,j]*z[i,j] for i in 1:nFactories for j in 1:nDemandPoint) +EQ)
println(M2)                

Min 14 x[1] + 14 x[2] + 14 x[3] + 2.49 z[1,1] + 5.21 z[1,2] + 3.76 z[1,3] + 4.85 z[1,4] + 2.07 z[1,5] + 1.46 z[2,1] + 2.54 z[2,2] + 1.83 z[2,3] + 1.86 z[2,4] + 4.76 z[2,5] + 3.26 z[3,1] + 3.08 z[3,2] + 2.6 z[3,3] + 3.76 z[3,4] + 4.45 z[3,5] + 0.005625 w[1,1] - 0.03375 y[1,1] + 0.005625 w[2,1] - 0.03375 y[2,1] + 0.005625 w[3,1] - 0.03375 y[3,1] + 0.005625 w[4,1] - 0.03375 y[4,1] + 0.005625 w[5,1] - 0.03375 y[5,1] + 0.0075 w[1,2] - 0.045 y[1,2] + 0.0075 w[2,2] - 0.045 y[2,2] + 0.0075 w[3,2] - 0.045 y[3,2] + 0.0075 w[4,2] - 0.045 y[4,2] + 0.0075 w[5,2] - 0.045 y[5,2] + 0.005625 w[1,3] - 0.03375 y[1,3] + 0.005625 w[2,3] - 0.03375 y[2,3] + 0.005625 w[3,3] - 0.03375 y[3,3] + 0.005625 w[4,3] - 0.03375 y[4,3] + 0.005625 w[5,3] - 0.03375 y[5,3] + 0.0075 w[1,4] - 0.045 y[1,4] + 0.0075 w[2,4] - 0.045 y[2,4] + 0.0075 w[3,4] - 0.045 y[3,4] + 0.0075 w[4,4] - 0.045 y[4,4] + 0.0075 w[5,4] - 0.045 y[5,4] + 0.010000000000000002 w[1,5] - 0.06000000000000001 y[1,5] + 0.010000000000000002 w[2,5] - 0.060000

In [7]:
defsolver(M2)

status = solve(M2)

Academic license - for non-commercial use only
Optimize a model with 2436 rows, 2448 columns and 7311 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-03, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 8e+02]
Presolve removed 1218 rows and 0 columns
Presolve time: 0.00s
Presolved: 1218 rows, 2448 columns, 6093 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.7830000e+04   4.936875e+03   0.000000e+00      0s
    1232   -1.0793000e+04   0.000000e+00   0.000000e+00      0s

Solved in 1232 iterations and 0.01 seconds
Optimal objective -1.079300000e+04


:Optimal

In [8]:
getvalue(x)

3-element Array{Float64,1}:
 500.0
 450.0
 470.0

In [9]:
getvalue(z)

3×5 Array{Float64,2}:
   0.0    0.0    0.0    0.0  500.0
 150.0    0.0    0.0  300.0    0.0
   0.0  100.0  270.0    0.0  100.0

In [10]:
getvalue(y)

5×243 Array{Float64,2}:
 150.0  150.0  150.0  150.0  150.0  …  150.0  150.0  150.0  150.0  150.0
 100.0  100.0  100.0  100.0  100.0     100.0  100.0  100.0  100.0  100.0
 250.0  250.0  250.0  250.0  250.0     270.0  270.0  270.0  270.0  270.0
 300.0  300.0  300.0  300.0  300.0     300.0  300.0  300.0  300.0  300.0
 600.0  600.0  600.0  600.0  600.0     600.0  600.0  600.0  600.0  600.0

## A multi-cuts L-shaped method

In [11]:
#to convert z[i,j] to x[i]
function convertZ2X(row,col)
    idx = (row - 1)* nDemandPoint+col+nFactories
    return idx
end
function convertX2Z(idx)
   row = Int(ceil((idx-nFactories)/nDemandPoint))
   col = mod(idx,nDemandPoint)
   return (row,col)
end
function convertW2Y(w_idx)
   idx = w_idx + nDemandPoint
   return idx
end

convertW2Y (generic function with 1 method)

In [12]:
function firststage()
    m = Model(solver=GurobiSolver(OutputFlag=0))
    
    @variable(m,x[1:nFactories+nFactories*nDemandPoint]>=0)
    @variable(m,θ[1:nScenario]>=-Inf)
    for i=1:nFactories
       @constraint(m,x[i]-sum(x[convertZ2X(i,j)] for j in 1:nDemandPoint)==0)
       @constraint(m,x[i]<=h[i])
    end
    initial_obj=sum(x[i]*c[i] for i in 1:nFactories) +sum(t[i,j]*x[convertZ2X(i,j)] for i in 1:nFactories for j in 1:nDemandPoint)
    @objective(m,Min,initial_obj)
    return m, x,θ,initial_obj
end

firststage (generic function with 1 method)

In [13]:
function add_regularization(m,ro,x,pre,prev_obj)
    obj=getobjective(m)-prev_obj
    alpha=(1/(2*ro))
    @objective(m,Min,obj+(alpha*sum((x[i]-pre[i])*(x[i]-pre[i]) for i in 1: length(pre) )))
    prev_obj=alpha*sum((x[i]-pre[i])*(x[i]-pre[i]) for i in 1: length(pre))
    return m,prev_obj
end

add_regularization (generic function with 1 method)

In [14]:
function secondstageCore(x, ξ)
    # ξ is the realized demands
     m = Model(solver=GurobiSolver(OutputFlag=0))
  
    @variable(m, y[1:2*nDemandPoint] >= 0)

    # We would gain to use sparse matrices
    T=zeros(2*nDemandPoint,nFactories+nFactories*nDemandPoint)
    for place in 1:nDemandPoint
        for factory in 1:nFactories 
             T[place+nDemandPoint,convertZ2X(factory,place)]=1
        end
    end
    T=-1*T
    h = [ξ; zeros(nDemandPoint)]
    return m, y, h, T
end 

secondstageCore (generic function with 1 method)

In [15]:
function secondstage(x, ξ)
    m, y, h, T = secondstageCore(x, ξ)
    
    @constraintref recourseConstraints[1:2*(nDemandPoint)]
   
    for j = 1:nDemandPoint
        # meet the demand   
        recourseConstraints[j] = @constraint(m, y[j]<=ξ[j] )
        #amount has to be equal to the transported
        recourseConstraints[j+nDemandPoint] = @constraint(m,y[j]+y[convertW2Y(j)] == sum(x[convertZ2X(i,j)] for i in 1:nFactories))
    end
    @objective(m, Min, sum(k[j]*y[convertW2Y(j)]-p[j]*y[j] for j in 1:nDemandPoint))
    return m, recourseConstraints, h, T
    
end

secondstage (generic function with 1 method)

In [16]:
function secondstagefeasibility(x, ξ)
    m, y, h, T = secondstageCore(x, ξ)

    
    @variable(m, slack_pos[1:2*nDemandPoint] >= 0)
    @variable(m, slack_neg[1:2*nDemandPoint] >= 0)
    @constraintref recourseConstraints[1:2*(nDemandPoint)]
    for j = 1:nDemandPoint
        # meet the demand   
        recourseConstraints[j] = @constraint(m, y[j]+slack_pos[j]-slack_neg[j] ==ξ[j] )
        #amount has to be equal to the transported
        recourseConstraints[j+nDemandPoint] = @constraint(m,y[j]+y[convertW2Y(j)] +slack_pos[j+nDemandPoint]-slack_neg[j+nDemandPoint] 
            == sum(x[convertZ2X(i,j)] for i in 1:nFactories))
    end
    @objective(m, Min, sum(slack_pos[j]+slack_neg[j] for j in 1:2*nDemandPoint))
    return m, recourseConstraints, h, T
end

secondstagefeasibility (generic function with 1 method)

In [17]:
function stop(Q, θ, k)
    nmax =200
    tol = 1e-12
    if (!(false in (θ.>= Q.-tol)) || (k == nmax))
    #if k==nmax
        return true
    else
        return false
    end
end

stop (generic function with 1 method)

In [18]:
function MultiCutLshaped(scenarios)
    nScenarios = size(scenarios)[1]
    prob=scenarios[:,nDemandPoint+1]
    pure_scenario=scenarios[:,1:nDemandPoint]

    k = 0       # iteration index
    nfcuts = 0  # number of feasibility cuts
    nocuts = zeros(nScenarios)  # number of optimality cuts
    first, x ,θ,initial_obj = firststage()
    n = length(x)
    Q = ones(nScenarios)*Inf
    valθ = ones(nScenarios)*(-Inf)
    while (!stop(Q, valθ, k))
        k += 1
        status = solve(first)
        println(first)
        if (status != :Optimal)
            println("Error: status ", status)
            println(first)
            return status, x, first
        end
        xsol = getvalue(x)
        println("\n-------θ = ",getvalue(θ),"\n")
        # Solve the second-stage programs
        for i = 1:nScenarios
            pr = prob[i]
            m, cstrs, h, T = secondstage(xsol, pure_scenario[i,:])
            status = solve(m);
            println(status)
            if (status == :Infeasible)
                # Build a feasibility cut
                m, cstrs, h, T = secondstagefeasibility(xsol, pure_scenario[i,:])
                solve(m)
                σ = getdual(cstrs)
                D = σ'*T
                d= σ'*h
                @constraint(first, sum(D[i]*x[i] for i in 1:nFactories+nFactories*nDemandPoint) >= d)
                nfcuts += 1
                break;
            elseif (status == :Optimal)
                Q[i] = pr*getobjectivevalue(m)
                π = getdual(cstrs)
                # Build the optimality cut component
                E = pr*(π'*T)
                e = pr*(dot(π,h))  #50
                if(nocuts[i]!=0)
                    valθ[i]=getvalue(θ[i])
                end
                if(nocuts[i]==0)
                    @constraint(first,dot(E,x)+θ[i]>=e)
                    obj_tmp=getobjective(first)
                    @objective(first, Min, obj_tmp+θ[i]) 
                    nocuts[i] += 1
                elseif (Q[i]>valθ[i])
                    @constraint(first,dot(E,x)+θ[i]>=e)
                    nocuts[i] += 1
                end  
            else
                println("Error second-stage resolution: status ", status)
                return status, x, first
            end
        end
    end
    
    println("Solved in ", k, " iterations.")
    return x, first,k
end

MultiCutLshaped (generic function with 1 method)

In [19]:
nScenario=size(Scenarios)[1]
x, firstst,iteration= MultiCutLshaped(Scenarios)

Academic license - for non-commercial use only
Min 14 x[1] + 14 x[2] + 14 x[3] + 2.49 x[4] + 5.21 x[5] + 3.76 x[6] + 4.85 x[7] + 2.07 x[8] + 1.46 x[9] + 2.54 x[10] + 1.83 x[11] + 1.86 x[12] + 4.76 x[13] + 3.26 x[14] + 3.08 x[15] + 2.6 x[16] + 3.76 x[17] + 4.45 x[18]
Subject to
 x[1] - x[4] - x[5] - x[6] - x[7] - x[8] == 0
 x[1] <= 500
 x[2] - x[9] - x[10] - x[11] - x[12] - x[13] == 0
 x[2] <= 450
 x[3] - x[14] - x[15] - x[16] - x[17] - x[18] == 0
 x[3] <= 650
 x[i] >= 0 for all i in {1,2,..,17,18}
 θ[i] for all i in {1,2,..,242,243}


-------θ = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic l



-------θ = [-54.0, -72.0, -54.0, -72.0, -96.0, -72.0, -54.0, -72.0, -54.0, -108.0, -144.0, -108.0, -144.0, -192.0, -144.0, -108.0, -144.0, -108.0, -54.0, -72.0, -54.0, -72.0, -96.0, -72.0, -54.0, -72.0, -54.0, -108.0, -144.0, -108.0, -144.0, -192.0, -144.0, -108.0, -144.0, -108.0, -216.0, -288.0, -216.0, -288.0, -384.0, -288.0, -216.0, -288.0, -216.0, -108.0, -144.0, -108.0, -144.0, -192.0, -144.0, -108.0, -144.0, -108.0, -54.0, -72.0, -54.0, -72.0, -96.0, -72.0, -54.0, -72.0, -54.0, -108.0, -144.0, -108.0, -144.0, -192.0, -144.0, -108.0, -144.0, -108.0, -54.0, -72.0, -54.0, -72.0, -96.0, -72.0, -54.0, -72.0, -54.0, -108.0, -144.0, -108.0, -144.0, -192.0, -144.0, -108.0, -144.0, -108.0, -216.0, -288.0, -216.0, -288.0, -384.0, -288.0, -216.0, -288.0, -216.0, -108.0, -144.0, -108.0, -144.0, -192.0, -144.0, -108.0, -144.0, -108.0, -216.0, -288.0, -216.0, -288.0, -384.0, -288.0, -216.0, -288.0, -216.0, -432.0, -576.0, -432.0, -576.0, -768.0, -576.0, -432.0, -576.0, -432.0, -216.0, -288.0

Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Ac

 -0.005625000000000001 x[4] + 0.03375 x[5] - 0.005625000000000001 x[6] + 0.03375 x[7] + 0.03375 x[8] - 0.005625000000000001 x[9] + 0.03375 x[10] - 0.005625000000000001 x[11] + 0.03375 x[12] + 0.03375 x[13] - 0.005625000000000001 x[14] + 0.03375 x[15] - 0.005625000000000001 x[16] + 0.03375 x[17] + 0.03375 x[18] + θ[171] >= -16.5375
 -0.011250000000000001 x[4] + 0.0675 x[5] - 0.011250000000000001 x[6] + 0.0675 x[7] + 0.0675 x[8] - 0.011250000000000001 x[9] + 0.0675 x[10] - 0.011250000000000001 x[11] + 0.0675 x[12] + 0.0675 x[13] - 0.011250000000000001 x[14] + 0.0675 x[15] - 0.011250000000000001 x[16] + 0.0675 x[17] + 0.0675 x[18] + θ[172] >= -34.650000000000006
 -0.015000000000000003 x[4] + 0.09000000000000002 x[5] - 0.015000000000000003 x[6] + 0.09000000000000002 x[7] + 0.09000000000000002 x[8] - 0.015000000000000003 x[9] + 0.09000000000000002 x[10] - 0.015000000000000003 x[11] + 0.09000000000000002 x[12] + 0.09000000000000002 x[13] - 0.015000000000000003 x[14] + 0.09000000000000002 x[1


-------θ = [-54.0, -72.0, -54.0, -72.0, -96.0, -72.0, -54.0, -72.0, -54.0, -108.0, -144.0, -108.0, -144.0, -192.0, -144.0, -108.0, -144.0, -108.0, -54.0, -72.0, -54.0, -72.0, -96.0, -72.0, -54.0, -72.0, -54.0, -108.0, -144.0, -108.0, -144.0, -192.0, -144.0, -108.0, -144.0, -108.0, -216.0, -288.0, -216.0, -288.0, -384.0, -288.0, -216.0, -288.0, -216.0, -108.0, -144.0, -108.0, -144.0, -192.0, -144.0, -108.0, -144.0, -108.0, -54.0, -72.0, -54.0, -72.0, -96.0, -72.0, -54.0, -72.0, -54.0, -108.0, -144.0, -108.0, -144.0, -192.0, -144.0, -108.0, -144.0, -108.0, -54.0, -72.0, -54.0, -72.0, -96.0, -72.0, -54.0, -72.0, -54.0, -108.0, -144.0, -108.0, -144.0, -192.0, -144.0, -108.0, -144.0, -108.0, -216.0, -288.0, -216.0, -288.0, -384.0, -288.0, -216.0, -288.0, -216.0, -108.0, -144.0, -108.0, -144.0, -192.0, -144.0, -108.0, -144.0, -108.0, -216.0, -288.0, -216.0, -288.0, -384.0, -288.0, -216.0, -288.0, -216.0, -432.0, -576.0, -432.0, -576.0, -768.0, -576.0, -432.0, -576.0, -432.0, -216.0, -288.0,

Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic license - for non-commercial use only
Optimal
Academic l

Excessive output truncated after 594731 bytes.

(x[i] >= 0 for all i in {1,2,..,17,18}, Minimization problem with:
 * 1510 linear constraints
 * 261 variables
Solver is Gurobi, 9)

In [20]:
getobjectivevalue(firstst)

-10792.999999999978

In [21]:
getvalue(x)

18-element Array{Float64,1}:
 500.0             
 450.0             
 469.9999999999999 
   0.0             
   0.0             
   0.0             
   0.0             
 500.0             
 149.99999999999994
   0.0             
   0.0             
 300.00000000000006
   0.0             
   0.0             
  99.99999999999984
 269.99999999999994
   0.0             
 100.00000000000009

In [22]:
iteration

9